#### Load/generate the graphs first

In [1]:
import numpy as np
import networkx as nx
import os


def make_dir(dir):
    if not os.path.exists(dir):
        os.mkdir(dir)



MIN_NODES = 10
MAX_NODES = 100

num_nodes = np.linspace(MIN_NODES, MAX_NODES, MAX_NODES//MIN_NODES)


dir = "networks/powerlaw_m=1_p=0.2"
os.makedirs(os.path.dirname(dir+"/"), exist_ok=True)

solver = "advantage"


try:
    Graphs = np.load(f"{dir}/graphs.npy", allow_pickle=True)
except:
    Graphs = np.empty(shape=(len(num_nodes),), dtype=object)
    for i, n in enumerate(num_nodes):
        Graphs[i] = nx.powerlaw_cluster_graph(n=n, m=1, p=0.2)
    np.save(f"{dir}/graphs.npy", Graphs)




dir = "output"
os.makedirs(os.path.dirname(dir+"/"), exist_ok=True)
make_dir(f"{dir}/{solver}")

#### Our standard "for loop"

In [5]:
from Qommunity.samplers.hierarchical.advantage_sampler import AdvantageSampler
from Qommunity.searchers.hierarchical_community_searcher import HierarchicalCommunitySearcher
from tqdm import tqdm


N_RUNS = 1

# Let's only check the graph of 100 nodes (embedding will take more time)
# as we remember the bigger the true problems were for graphs of 70+ nodes
big_graph_idx = 9


def run():
    mods = np.zeros((num_nodes[big_graph_idx:].shape[0], N_RUNS))
    comms = np.empty((num_nodes[big_graph_idx:].shape[0], N_RUNS), dtype=object)

    mods_graph_N = np.zeros((N_RUNS))
    comms_graph_N = np.empty((N_RUNS), dtype=object)
    for i, G in tqdm(enumerate(Graphs[big_graph_idx:])):
        advantage = AdvantageSampler(G, num_reads=100)
        hierch_searcher = HierarchicalCommunitySearcher(advantage)
        
        current_net_size = int(G.number_of_nodes())
        for r in range(N_RUNS):
            comms_res = hierch_searcher.hierarchical_community_search()
            mod_score = nx.community.modularity(G, comms_res)
            mods_graph_N[r] = mod_score
            comms_graph_N[r] = comms_res

            try:
                np.save(f"{dir}/{solver}/{solver}-network_size_{current_net_size}", mods_graph_N)
                np.save(f"{dir}/{solver}/{solver}-network_size_{current_net_size}_comms", comms_graph_N)
            except Exception as e:
                print(f"iter {i} run {r} npy saving failed:\n{e}")

        mods[i] = mods_graph_N
        comms[i] = comms_graph_N
        np.save(f"{dir}/{solver}/{solver}-network_size_{current_net_size}", mods_graph_N)
        np.save(f"{dir}/{solver}/{solver}-network_size_{current_net_size}_comms", comms_graph_N)

    np.save(f"{dir}/{solver}/{solver}-mods_final", mods)
    np.save(f"{dir}/{solver}/{solver}-comms_final", comms)

In [8]:
%load_ext line_profiler

from QHyper.solvers.quantum_annealing.advantage import Advantage, OptimizerFunction

%lprun -f OptimizerFunction.run_advantage run()

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


1it [02:10, 130.94s/it]


Timer unit: 1e-07 s

Total time: 96.4437 s
File: C:\Users\basia\Desktop\Praca_Inzynierska_2024\QHyper\QHyper\solvers\quantum_annealing\advantage.py
Function: run_advantage at line 51

Line #      Hits         Time  Per Hit   % Time  Line Contents
    51                                               def run_advantage(self, args: npt.NDArray) -> SampleSet:
    52        21     344659.0  16412.3      0.0          qubo = Converter.create_qubo(self.problem, args)
    53        21     268365.0  12779.3      0.0          qubo_terms, offset = convert_qubo_keys(qubo)
    54        21     572979.0  27284.7      0.1          bqm = BinaryQuadraticModel.from_qubo(qubo_terms, offset=offset)
    55        42  963250652.0    2e+07     99.9          sampleset = self.embedding_compose.sample(
    56        21        127.0      6.0      0.0              bqm, num_reads=self.num_reads, chain_strength=self.chain_strength
    57                                                   )
    58                      